<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/Deformable_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deformable Style Transfer (ECCV 2020)

![](https://sunniesuhyoung.github.io/DST-page/imgs/figure1.png)

In this notebook, we demonstrate deformable style transfer (DST) using STROTSS as the base style transfer method.

Project page: https://sunniesuhyoung.github.io/DST-page/  
Paper: https://arxiv.org/abs/2003.11038  
Code: https://github.com/sunniesuhyoung/DST  
1min video: https://youtu.be/7qUzfcn6TPk  
10min talk: https://youtu.be/mVU5tSxS4is

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Подготовка необходимых компонентов</font></b>
resul_image_size = "256" #@param ["256", "512", "1024"]
!git clone https://github.com/sunniesuhyoung/DST
import os
os.chdir('DST')

import matplotlib.pyplot as plt
import numpy as np
import torch
from utils_misc import pil_loader, pil_resize_long_edge_to, pil_to_tensor
from IPython.display import clear_output

# Image will be resized to have a long side of im_side
%env im_size = int(resul_image_size)

# If you choose cuda, make sure you select GPU under [Runtime]-[Change runtime type]
%env device = cuda

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Загрузка файлов</font></b>
'''%env content_path = example/content.jpg
%env style_path = example/style.jpg #'''

from google.colab import files
print('Загрузите фото, на который переносим стиль:')
content_im = files.upload()
print('\nЗагрузите фото, откуда берем стиль:')
style_im = files.upload()
os.environ['content_path'] = list(content_im)[0]
os.environ['style_path'] = list(style_im)[0] #'''

# Load and resize input images
content_pil = pil_resize_long_edge_to(pil_loader(os.environ['content_path']), int(os.environ['im_size']))
width, height = content_pil.size
style_pil = pil_resize_long_edge_to(pil_loader(os.environ['style_path']), int(os.environ['im_size']))
content_im_orig = pil_to_tensor(content_pil).to(os.environ['device'])
style_im_orig = pil_to_tensor(style_pil).to(os.environ['device'])

clear_output()
fig = plt.figure(figsize=(8, 6))
plt.subplot(1, 2, 1)
plt.imshow(content_pil); plt.axis('off'); plt.title('Content image');
plt.subplot(1, 2, 2)
plt.imshow(style_pil); plt.axis('off'); plt.title('Style image');

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Нахождение ключевых точек</font></b>
%env pts_path = example/NBBresults
%env content_pts_path = example/NBBresults/correspondence_A.txt
%env style_pts_path = example/NBBresults/correspondence_B.txt
%env activation_path = example/NBBresults/correspondence_activation.txt
%env output_path = example/CleanedPts
%env NBB = 1
%env max_num_points = 80
%env b = 10
%env content_marked = example/CleanedPts/A_selected_final.png
%env style_marked = example/CleanedPts/B_selected_final.png

!python NBB/main.py --results_dir ${pts_path} --imageSize ${im_size} --fast \
  --datarootA ${content_path} --datarootB ${style_path}

!python cleanpoints.py ${content_path} ${style_path} ${content_pts_path} ${style_pts_path} \
  ${activation_path} ${output_path} ${im_size} ${NBB} ${max_num_points} ${b}

content_marked_pil = pil_resize_long_edge_to(pil_loader(os.environ['content_marked']), int(os.environ['im_size']))
style_marked_pil = pil_resize_long_edge_to(pil_loader(os.environ['style_marked']), int(os.environ['im_size']))

clear_output()
fig = plt.figure(figsize=(8, 6))
plt.subplot(1, 2, 1)
plt.imshow(content_marked_pil); plt.axis('off'); plt.title('Content image with points');
plt.subplot(1, 2, 2)
plt.imshow(style_marked_pil); plt.axis('off'); plt.title('Style image with points');

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Перенос стиля</font></b>
%env content_pts_path = example/CleanedPts/correspondence_A.txt
%env style_pts_path = example/CleanedPts/correspondence_B.txt
%env output_dir = example/DSTresults
%env output_prefix = example
%env max_iter = 250
%env checkpoint_iter = 50
%env content_weight = 8
%env warp_weight = 0.5
%env reg_weight = 50
%env optim = sgd
%env lr = 0.2
%env verbose = 0
%env save_intermediate = 0
%env save_extra = 0
%env result_path = example/DSTresults/example.png

!python -W ignore main.py ${content_path} ${style_path} ${content_pts_path} ${style_pts_path} \
  ${output_dir} ${output_prefix} ${im_size} ${max_iter} \
  ${checkpoint_iter} ${content_weight} ${warp_weight} ${reg_weight} ${optim} \
  ${lr} ${verbose} ${save_intermediate} ${save_extra} ${device}

result_pil = pil_resize_long_edge_to(pil_loader(os.environ['result_path']), int(os.environ['im_size']))

clear_output()
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(content_pil); plt.axis('off'); plt.title('Content image');
plt.subplot(1, 3, 2)
plt.imshow(style_pil); plt.axis('off'); plt.title('Style image');
plt.subplot(1, 3, 3)
plt.imshow(result_pil); plt.axis('off'); plt.title('DST output');

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Показать результат</font></b>
result_pil